In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,Bidirectional,SimpleRNN,TimeDistributed, Dense

In [2]:
#example
#input_text = [
 #   "quick brown fox jumps",
  #  "brown fox jumps over",
   # "fox jumps over the",
    #"jumps over the lazy",
    #"over the lazy dog"
#]

input_text = [
    "The sun is shining brightly",
    "shining brightly in the blue",
    "brightly in the blue sky",
    "in the blue sky with",
    "the blue sky with few",
    "blue sky with few fluffy",
    "sky with few fluffy clouds"
]

In [3]:
# Tokenize input text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(input_text)
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

Vocabulary size: 13


In [4]:
# Convert input text to sequences
input_sequences = tokenizer.texts_to_sequences(input_text)

In [5]:
# Pad input sequences
max_len = max(len(seq) for seq in input_sequences)
padded_sequences = pad_sequences(input_sequences, maxlen=max_len, padding='post')
print("Padded input sequences:", padded_sequences)

Padded input sequences: [[ 1 10 11  8  5]
 [ 8  5  6  1  2]
 [ 5  6  1  2  3]
 [ 6  1  2  3  4]
 [ 1  2  3  4  7]
 [ 2  3  4  7  9]
 [ 3  4  7  9 12]]


In [6]:
# Define Bi-RNN model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=10, input_length=max_len-1),
    Bidirectional(SimpleRNN(64, return_sequences=True)),
    Bidirectional(SimpleRNN(64, return_sequences=True)),
    TimeDistributed(Dense(vocab_size, activation='softmax'))
])

In [7]:
# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 10)             130       
                                                                 
 bidirectional (Bidirectiona  (None, 4, 128)           9600      
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 4, 128)           24704     
 nal)                                                            
                                                                 
 time_distributed (TimeDistr  (None, 4, 13)            1677      
 ibuted)                                                         
                                                                 
Total params: 36,111
Trainable params: 36,111
Non-trainable params: 0
___________________________________________________

In [8]:
# Train the model
X_data = padded_sequences[:, :-1]  # Input data (remove last word from each sequence)
y_data = padded_sequences[:, 1:]   # Output data (remove first word from each sequence)
model.fit(X_data, y_data, epochs=50, verbose=1)

Epoch 1/50


1/1 [==============================] - 8s 8s/step - loss: 2.5684 - accuracy: 0.1071
Epoch 2/50
1/1 [==============================] - 0s 26ms/step - loss: 2.5248 - accuracy: 0.2143
Epoch 3/50
1/1 [==============================] - 0s 17ms/step - loss: 2.4807 - accuracy: 0.2857
Epoch 4/50
1/1 [==============================] - 0s 16ms/step - loss: 2.4352 - accuracy: 0.4286
Epoch 5/50
1/1 [==============================] - 0s 17ms/step - loss: 2.3873 - accuracy: 0.5714
Epoch 6/50
1/1 [==============================] - 0s 16ms/step - loss: 2.3363 - accuracy: 0.6429
Epoch 7/50
1/1 [==============================] - 0s 32ms/step - loss: 2.2815 - accuracy: 0.6429
Epoch 8/50
1/1 [==============================] - 0s 16ms/step - loss: 2.2226 - accuracy: 0.6786
Epoch 9/50
1/1 [==============================] - 0s 16ms/step - loss: 2.1591 - accuracy: 0.6786
Epoch 10/50
1/1 [==============================] - 0s 16ms/step - loss: 2.0910 - accuracy: 0.6786
Epoch 11/50
1/1 [============

In [9]:
# Print input sequences and padded input sequences
print("Input sequences:", input_text)
print("Padded input sequences:", padded_sequences)

# Verify vocabulary size and index-word mapping
print("Vocabulary size:", vocab_size)
print("Index-word mapping:", tokenizer.index_word)

# Inspect the model's architecture
print(model.summary())

Input sequences: ['The sun is shining brightly', 'shining brightly in the blue', 'brightly in the blue sky', 'in the blue sky with', 'the blue sky with few', 'blue sky with few fluffy', 'sky with few fluffy clouds']
Padded input sequences: [[ 1 10 11  8  5]
 [ 8  5  6  1  2]
 [ 5  6  1  2  3]
 [ 6  1  2  3  4]
 [ 1  2  3  4  7]
 [ 2  3  4  7  9]
 [ 3  4  7  9 12]]
Vocabulary size: 13
Index-word mapping: {1: 'the', 2: 'blue', 3: 'sky', 4: 'with', 5: 'brightly', 6: 'in', 7: 'few', 8: 'shining', 9: 'fluffy', 10: 'sun', 11: 'is', 12: 'clouds'}
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 10)             130       
                                                                 
 bidirectional (Bidirectiona  (None, 4, 128)           9600      
 l)                                                              
                                      

In [10]:
# Function to generate next word prediction
def predict_next_word(input_text):
    input_seq = tokenizer.texts_to_sequences([input_text])
    padded_input_seq = pad_sequences(input_seq, maxlen=max_len-1, padding='post')
    predicted_probs = model.predict(padded_input_seq)
    predicted_index = np.argmax(predicted_probs[0][-1])
    predicted_word = tokenizer.index_word.get(predicted_index, "<Unknown>")
    return predicted_word

In [11]:
# Generate next word predictions
for text in input_text:
    predicted_word = predict_next_word(text)
    print(f"Input: {text}, Predicted next word: {predicted_word}")

1/1 [==============================] - 1s 1s/step
Input: The sun is shining brightly, Predicted next word: clouds
1/1 [==============================] - 0s 62ms/step
Input: shining brightly in the blue, Predicted next word: sky
1/1 [==============================] - 0s 47ms/step
Input: brightly in the blue sky, Predicted next word: with
1/1 [==============================] - 0s 63ms/step
Input: in the blue sky with, Predicted next word: few
1/1 [==============================] - 0s 87ms/step
Input: the blue sky with few, Predicted next word: fluffy
1/1 [==============================] - 0s 64ms/step
Input: blue sky with few fluffy, Predicted next word: clouds
1/1 [==============================] - 0s 47ms/step
Input: sky with few fluffy clouds, Predicted next word: clouds
